# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../WeatherPy/output_Data/cities.csv', encoding="utf-8")
cities_df.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Rikitea,11,PF,1651975282,81,-23.1203,-134.9692,77.54,15.88
1,Saint-Joseph,19,RE,1651975282,80,-21.3667,55.6167,68.38,11.12
2,Saint George,0,US,1651975244,14,37.1041,-113.5841,86.63,27.63
3,Mataura,100,NZ,1651975283,96,-46.1927,168.8643,58.48,2.01
4,Cape Town,0,ZA,1651975022,80,-33.9258,18.4232,57.90,14.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Fill Humidity values and convert to float
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
humidity = cities_df["Humidity"].tolist()
markers = gmaps.marker_layer(locations,info_box_content=[f"Humidity: {humidity}" for humidity in humidity])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Preferred City - Max Temp Greater than 60 and less than 80, Humidity less than 75, Cloudiness less than 30%
pref_cities = cities_df.loc[
    (cities_df["Max Temp"] > 60) &
    (cities_df["Max Temp"] < 80) &
    (cities_df["Humidity"] < 75) & 
    (cities_df["Cloudiness"] < 30)
]
pref_cities = pref_cities.dropna(how='any')
pref_cities.reset_index(inplace=True)
del pref_cities['index']

# Display to output
pref_cities.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Vaini,27,TO,1651975285,73,-21.2000,-175.2000,75.47,12.44
1,Shingū,3,JP,1651975302,53,33.7333,135.9833,72.46,4.94
2,Barra do Bugres,0,BR,1651975302,62,-15.0725,-57.1811,65.73,0.65
3,Ohangaron,0,UZ,1651975303,47,40.9064,69.6383,69.94,5.77
4,Springfield,0,US,1651975297,64,37.2153,-93.2982,66.06,11.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(pref_cities)

In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
     "radius": 5000, 
     "keyword":"hotel",
     "key": g_key,
}

In [11]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row["Latitude"]
    long = row["Longitude"]
    city_name = row["City Name"]

    # add keyword to params dict
    params["location"] = f"{lat},{long}"

    # assemble url and make API request
    print(f"Retrieving Closest Hotel for Index# {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index,"Hotel Name"] = results[0]['name']
          
    except (KeyError, IndexError):
        print("No Hotel Found")
        
    print("---------------")
    
    
print("---End of Search---")

Retrieving Closest Hotel for Index# 0: Vaini.
Closest hotel in Vaini is Keleti Beach Resort.
---------------
Retrieving Closest Hotel for Index# 1: Shingū.
Closest hotel in Shingū is Shingu UI Hotel.
---------------
Retrieving Closest Hotel for Index# 2: Barra do Bugres.
Closest hotel in Barra do Bugres is American Palace Hotel.
---------------
Retrieving Closest Hotel for Index# 3: Ohangaron.
Closest hotel in Ohangaron is Hotel.
---------------
Retrieving Closest Hotel for Index# 4: Springfield.
Closest hotel in Springfield is Holiday Inn & Suites Springfield - I-44, an IHG Hotel.
---------------
Retrieving Closest Hotel for Index# 5: Le Port.
Closest hotel in Le Port is Lodge Roche Tamarin & Spa Village Nature.
---------------
Retrieving Closest Hotel for Index# 6: Los Llanos de Aridane.
Closest hotel in Los Llanos de Aridane is Hotel Hacienda de Abajo.
---------------
Retrieving Closest Hotel for Index# 7: Buraidah.
Closest hotel in Buraidah is Best Western Plus Buraidah.
----------

In [12]:
hotel_df.rename(columns={"City Name" : "City"}, inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Vaini,27,TO,1651975285,73,-21.2000,-175.2000,75.47,12.44,Keleti Beach Resort
1,Shingū,3,JP,1651975302,53,33.7333,135.9833,72.46,4.94,Shingu UI Hotel
2,Barra do Bugres,0,BR,1651975302,62,-15.0725,-57.1811,65.73,0.65,American Palace Hotel
3,Ohangaron,0,UZ,1651975303,47,40.9064,69.6383,69.94,5.77,Hotel
4,Springfield,0,US,1651975297,64,37.2153,-93.2982,66.06,11.50,"Holiday Inn & Suites Springfield - I-44, an IH..."
5,Le Port,2,RE,1651975308,72,-20.9373,55.2919,73.36,7.49,Lodge Roche Tamarin & Spa Village Nature
6,Los Llanos de Aridane,20,ES,1651975312,60,28.6585,-17.9182,65.12,12.66,Hotel Hacienda de Abajo
7,Buraidah,3,SA,1651975313,23,26.3260,43.9750,70.81,11.41,Best Western Plus Buraidah
8,Opotiki,4,NZ,1651975315,64,-38.0833,177.2833,67.39,5.79,NaN
9,San Matías,0,BO,1651975316,48,-16.3667,-58.4000,69.80,2.93,Hotel Cochabamba. .


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
plt.savefig('plot.png')

<Figure size 432x288 with 0 Axes>